In [ ]:
using DataFrames, CSV, GLM, MLJ, VegaLite

In [ ]:
# Import training dataset
lichen_training = CSV.read("C:/Users/julio/Downloads/lichen_training.csv", DataFrame) |>
x -> select(x, "Total.lichen", "basal_area", "PL", "mean_Long") |>
x -> rename(x, ["biomass", "basalarea", "pl", "lon"]) |>
x -> coerce(x, Count => Continuous);

In [ ]:
MLJ.schema(lichen_training)

In [ ]:
# Split predictors (X) and response variable (y)
y, X = unpack(lichen_training, ==(:biomass), _ -> true; rng=1010);

In [ ]:
# Load a LinearRegressor using the GLM package
LinearRegressor = @load LinearRegressor pkg=GLM add=true verbosity=0

In [ ]:
# Instantiate a linear regressor model using a pipeline
LinearRegressorPipe = @pipeline(Standardizer(), LinearRegressor())

In [ ]:
# Create an MLJ machine (model + data)
LinearModel = machine(LinearRegressor(), X, y)

In [ ]:
# Split into train/test datasets
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1010)

In [ ]:
# Fit the machine
fit!(LinearModel, X[train])

In [ ]:
# Check trained parameters
fitted_params(LinearModel)

In [ ]:
# Import CSV to dataframe, select some columns
lichen = CSV.read("C:/Users/julio/Downloads/predicted_biomass_Nov2021.csv", DataFrame; missingstring="") |>
x -> select(x, :BASAL_AREA, :PL, :LON) |>
x -> rename(x, ["basalarea", "pl", "lon"]);

In [ ]:
biomass = predict(LinearModel, lichen)